# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
city_api_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_api_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Rikitea,PF,-23.12,-134.97,72.72,77,43,9.53
1,Busselton,AU,-33.65,115.33,72.95,30,73,13.96
2,Hithadhoo,MV,-0.60,73.08,82.27,59,70,1.57
3,Puerto Ayora,EC,-0.74,-90.35,68.00,86,100,5.99
4,Amapá,BR,1.00,-52.00,69.48,98,13,2.26


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [30]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_api_df[["Latitude", "Longitude"]]
# Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=city_api_df["Humidity (%)"])
# Add the heatmap layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# Temps b/w 60-70
# Humidity below 50%
temps = city_api_df[(city_api_df["Temperature (F)"] > 60) & (city_api_df["Temperature (F)"] < 70)]["Temperature (F)"]
humid = city_api_df[(city_api_df["Humidity (%)"] < 50)]["Humidity (%)"]
my_weather_df = city_api_df[(city_api_df["Temperature (F)"] > 60) & (city_api_df["Temperature (F)"] < 70) &(city_api_df["Humidity (%)"] < 50)]
my_weather_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
89,Flinders,AU,-34.58,150.86,66.99,36,3,15.99
131,Korem,ET,12.51,39.52,65.16,46,100,2.19
159,Elko,US,41.00,-115.50,60.31,33,1,4.12
161,Alamogordo,US,32.90,-105.96,67.59,28,1,4.70
189,Yangi Marg`ilon,UZ,40.43,71.72,69.80,26,0,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df = my_weather_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
89,Flinders,AU,-34.58,150.86,66.99,36,3,15.99,
131,Korem,ET,12.51,39.52,65.16,46,100,2.19,
159,Elko,US,41.00,-115.50,60.31,33,1,4.12,
161,Alamogordo,US,32.90,-105.96,67.59,28,1,4.70,
189,Yangi Marg`ilon,UZ,40.43,71.72,69.80,26,0,2.24,


In [33]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")

Shellharbour Resort & Conference Centre
Hotel not found... skipping.
Hotel not found... skipping.
Fairfield Inn & Suites by Marriott Alamogordo
Ikathouse
La Quinta Inn & Suites by Wyndham Great Falls
Oyster Bay House Rental
Faroja City Iyindeko Cas-jay
Hotel not found... skipping.
Apartment on Karla Libknekhta 17
Hotel not found... skipping.
Otel' " Ryzhiy Pos"
NUR
Buyant Hotel
Crowne Plaza Ordos
الاستاذة مروى اليوسفي
Boambee Waters Caravan Park
Hotel not found... skipping.
Willowbend Campground & Cabins
Lakeview guesthouse


In [34]:
hotel_df.reset_index()
hotel_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
89,Flinders,AU,-34.58,150.86,66.99,36,3,15.99,Shellharbour Resort & Conference Centre
131,Korem,ET,12.51,39.52,65.16,46,100,2.19,
159,Elko,US,41.00,-115.50,60.31,33,1,4.12,
161,Alamogordo,US,32.90,-105.96,67.59,28,1,4.70,Fairfield Inn & Suites by Marriott Alamogordo
189,Yangi Marg`ilon,UZ,40.43,71.72,69.80,26,0,2.24,Ikathouse


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [36]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))